In [49]:

import numpy as np 
import pandas as pd 
from datetime import datetime



import seaborn as sns
sns.set(rc={'figure.figsize':(10,6)})
custom_colors = ["#4e89ae", "#c56183","#ed6663","#ffa372"]


import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg

#Colorama
from colorama import Fore, Back, Style
y_= Fore.CYAN
m_= Fore.WHITE


import networkx as nx
import plotly.graph_objects as go 


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


df=pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')



In [50]:
print(f"{m_}Total no. of records:{y_}{df.shape}\n")
print(f"{m_}Corresponding Data types of data columns: \n{y_}{df.dtypes}")

In [51]:
df['Timestamp'] = [datetime.fromtimestamp(i) for i in df['Timestamp']]
df = df.set_index('Timestamp')
df = df.resample("24H").mean()
df.head()

In [52]:
values_missed = pd.DataFrame()
values_missed['column'] = df.columns

values_missed['percent'] = [round(100* df[z].isnull().sum() / len(df), 2) for z in df.columns]
values_missed = values_missed.sort_values('percent',ascending=False)
values_missed = values_missed[values_missed['percent']>0]

fig = sns.barplot(
    x=values_missed['percent'], 
    y=values_missed["column"], 
    orientation='horizontal',palette="winter"
).set_title('Missed values percent for every column')

In [53]:
def filling_the_misssing_values(df):
    df['Open'] = df['Open'].interpolate()
    df['Close'] = df['Close'].interpolate()
    df['Weighted_Price'] = df['Weighted_Price'].interpolate()
    df['Volume_(BTC)'] = df['Volume_(BTC)'].interpolate()
    df['Volume_(Currency)'] = df['Volume_(Currency)'].interpolate()
    df['High'] = df['High'].interpolate()
    df['Low'] = df['Low'].interpolate()
    print(f'{m_}After interploation,No. of Missing values:\n{y_}{df.isnull().sum()}')
    
filling_the_misssing_values(df)

In [54]:
df.columns

In [55]:
naya_df=df.groupby('Timestamp').mean()
naya_df=naya_df[['Volume_(BTC)', 'Close','Volume_(Currency)']]
naya_df.rename(columns={'Volume_(BTC)':'Volume_market_mean','Close':'close_mean','Volume_(Currency)':'volume_curr_mean'},inplace=True)
naya_df.head()

In [56]:
merged_df = df.merge(naya_df, left_on='Timestamp',
                                right_index=True)
merged_df['volume(BTC)/Volume_market_mean'] = merged_df['Volume_(BTC)'] / merged_df['Volume_market_mean']
merged_df['Volume_(Currency)/volume_curr_mean'] = merged_df['Volume_(Currency)'] / merged_df['volume_curr_mean']

merged_df['close/close_market_mean'] = merged_df['Close'] / merged_df['close_mean']
merged_df['open/close'] = merged_df['Open'] / merged_df['Close']
merged_df["gap"] = merged_df["High"] - merged_df["Low"] 
merged_df.head()

In [57]:

merged_df.info()


In [58]:
def tigna_plot(x, title,c):
    fig, ax = plt.subplots(3,1,figsize=(25,10),sharex=True)
    sns.distplot(x, ax=ax[0],color=c)
    ax[0].set(xlabel=None)
    ax[0].set_title('Histogram + KDE')
    sns.boxplot(x, ax=ax[1],color=c)
    ax[1].set(xlabel=None)
    ax[1].set_title('Boxplot')
    sns.violinplot(x, ax=ax[2],color=c)
    ax[2].set(xlabel=None)
    ax[2].set_title('Violin plot')
    fig.suptitle(title, fontsize=30)
    plt.tight_layout(pad=3.0)
    plt.show()

In [59]:
tigna_plot(df['Open'],'Distribution of Opening price',custom_colors[0])

In [60]:
tigna_plot(df['High'],'Distribution of the highest price',custom_colors[1])

In [61]:
tigna_plot(df['Low'],'Distribution of Lowest Price',custom_colors[2])

In [62]:
tigna_plot(df['Close'],'Distribution of the closing Price',custom_colors[3])

In [63]:
tigna_plot(df['Volume_(BTC)'],'Distribution of Volume in BTC ',custom_colors[0])

In [64]:
tigna_plot(df['Volume_(Currency)'],'Distribution of Volume',custom_colors[1])

In [65]:
tigna_plot(df['Weighted_Price'],'Distribution of Weighted price',custom_colors[2])

In [66]:
plt.figure(figsize=(8,8))
corr=merged_df[merged_df.columns[1:]].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(merged_df[merged_df.columns[1:]].corr(), mask=mask, cmap='coolwarm', vmax=.3, center=0,
            square=True, linewidths=.5,annot=True)
plt.savefig('my_image.png')
plt.show()

In [67]:
merged_df=merged_df.drop(columns=['volume(BTC)/Volume_market_mean','Volume_(Currency)/volume_curr_mean','close/close_market_mean'])
merged_df.columns

In [78]:
indices = corr.index.values
cor_matrix = np.asmatrix(corr)
G = nx.from_numpy_matrix(cor_matrix)
G = nx.relabel_nodes(G,lambda x: indices[x])
#G.edges(data=True)


def corr_network(g, co_dir, min_cor):
    h = g.copy()

    for i, j, w in g.edges(data=True):       
        if co_dir == "positive":
            if w["weight"] < 0 or w["weight"] < min_cor:
                h.remove_edge(i, j)
        else:
            if w["weight"] >= 0 or w["weight"] > min_cor:
                h.remove_edge(i, j)
                
    e,w = zip(*nx.get_edge_attributes(h,'weight').items())
    w = tuple([(1+abs(i))**2 for i in w])
   
    j = dict(nx.degree(h))
    nodelist=j.keys()
    node_sizes=j.values()
    
    positions=nx.circular_layout(h)
    
    plt.figure(figsize=(5,5))

    nx.draw_networkx_nodes(h,positions,node_color='#d100d1',nodelist=nodelist,
                       node_size=tuple([x**2 for x in node_sizes]),alpha=0.8)

    nx.draw_networkx_labels(h, positions, font_size=13)

    if co_dir == "positive":
        edge_colour = plt.cm.summer 
    else:
        edge_colour = plt.cm.autumn
        
    nx.draw_networkx_edges(h, positions, edgelist=e,style='solid',
                          width=w, edge_color = w, edge_cmap = edge_colour,
                          edge_vmin = min(w), edge_vmax=max(w))
    plt.axis("off")
    plt.show() 

In [79]:
corr_network(G, co_dir="positive",min_cor = 0.5)
